### **Step-0:** `Setup`

Impor pustaka yang dibutuhkan dalam analisis

In [61]:
import random
import pandas as pd 
import altair as alt

import warnings
warnings.filterwarnings('ignore')

Load data dan simpan dalam variabel `df` sebagai dataframe

In [62]:
raw = pd.read_excel('online retail.xlsx', sheet_name='Year 2010-2011')
raw.sample()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
86589,543555,21314,SMALL GLASS HEART TRINKET POT,8,2011-02-10 10:18:00,2.1,17686.0,United Kingdom


### **Setup-1:** `Skimming Data`

In [63]:
df = raw.copy()

Langkah awal, cek format data dan anomali:

In [64]:
print(f'Jumlah baris dan kolom: {df.shape}')

pd.DataFrame(
    {
    'feature': df.columns.values,
    'data_type': df.dtypes.values,
    'null_value': df.isna().mean().values * 100,
    'n_unique': df.nunique().values,
    'sample_unique': [df[col].unique() for col in df.columns]
    }
)

Jumlah baris dan kolom: (541910, 8)


,feature,data_type,null_value,n_unique,sample_unique
0,Invoice,object,0.000000,25900,"[536365, 536366, 536368, 536367, 536369, 53637..."
1,StockCode,object,0.000000,4070,"[85123A, 71053, 84406B, 84029G, 84029E, 22752,..."
2,Description,object,0.268310,4223,"[WHITE HANGING HEART T-LIGHT HOLDER, WHITE MET..."
3,Quantity,int64,0.000000,722,"[6, 8, 2, 3, 32, 4, 24, 12, 48, 18, 20, 36, 80..."
4,InvoiceDate,datetime64[ns],0.000000,23260,"[2010-12-01T08:26:00.000000000, 2010-12-01T08:..."
5,Price,float64,0.000000,1630,"[2.55, 3.39, 2.75, 7.65, 4.25, 1.85, 4.95, 1.6..."
6,Customer ID,float64,24.926648,4372,"[17850.0, 13047.0, 12583.0, 13748.0, 15100.0, ..."
7,Country,object,0.000000,38,"[United Kingdom, France, Australia, Netherland..."


__Insight :__

* Data terdiri dari 541910 baris dan 8 kolom.
* Missing value pada variabel `description` (25%) dan `customer id` (0.26%).
* Tipe data variabel `customer id` tidak sesuai.

### **Step-2:** `Data Cleaning`

Ubah tipe data dari kolom `Customer ID` menjadi integer.

In [65]:
df['Customer ID'] = df['Customer ID'].astype('Int64')

Nilai negatif pada kolom `Quantity` bisa kita tangani dengan membuatnya positif

In [66]:
df.Quantity = df.Quantity.apply(abs)

Nilai nol atau negative pada kolom `Price` kita tangani dengan memisahkannya

In [67]:
df = df[df.Price > 0]

Missing value pada kolom `Description` bisa digantikan dengan string **OTHER**

In [68]:
df.fillna({'Description': 'OTHER'}, inplace=True)

Kita exclude kan barang dengan deskripsi yang tidak valid

In [69]:
df = df[~df.Description.isin(['OTHER', 'Manual', 'check'])]

Kita pisahkan untuk dataframe dengan nan value simpan dalam `df2`

In [70]:
df2 = df[df["Customer ID"].isna()]

Sedangkan dataframe selain nan value lalu simpan sebagai `df3`.

In [71]:
df3 = df[~df["Customer ID"].isna()]

Validasi jika Customer ID yang bernilai `<NA>` apakah memiliki invoice terdaftar di `df3` atau tidak. Jika ada, maka nilai `<NA>` akan di gantikan dengan Customer ID yang berkaitan, namun jika sebaliknya, maka asumsi "mereka adalah customer baru" terbukti dan bisa digunakan.

In [72]:
pd.merge(left=df2, right=df3, on='Invoice', how='inner')

,Invoice,StockCode_x,Description_x,Quantity_x,InvoiceDate_x,Price_x,Customer ID_x,Country_x,StockCode_y,Description_y,Quantity_y,InvoiceDate_y,Price_y,Customer ID_y,Country_y


Kita hitung berapa jumlah nilai unik dari Invoice pada `df2`. Ini berguna untuk memastikan berapa banyak variasi bilangan random yang digunakan.

In [73]:
df2.Invoice.nunique()

1540

Bangkitkan bilangan random untuk `Customer ID` berbeda sejumlah `Invoice`, dan pastikan satu nilai random untuk satu `Customer ID`.

In [74]:
new_customer_id = random.sample(range(df['Customer ID'].max(), df['Customer ID'].max() + 10000), 2000)

Input kan bilangan acak tersebut ke `df2`.

In [75]:
id = {}
for i, val in enumerate(df2.Invoice.value_counts().index):
    if val not in id.keys():
        id.update(
            {
                val: new_customer_id[i]
            }
        )
    else:
        continue

df2['Customer ID'] = df2.Invoice.apply(lambda x: id[x])

Validasi ulang untuk memastikan jumlah unique value untuk Customer ID yang baru sesuai dengan unique value Invoice.

In [76]:
df2['Customer ID'].nunique()

1540

Gabungkan antara `df2` dengan `df3`, lalu simpan dalam variabel `df_merge`

In [77]:
df_merge = pd.concat([df2, df3], axis=0, ignore_index=True)

Terakhir cek data duplikat, dan terapkan `drop_duplicates` untuk menghapus salah satu nya:

In [78]:
print(f'Jumlah data duplikat: {df_merge[df_merge.duplicated()].shape[0]}')
df_merge.drop_duplicates(keep='first', inplace=True, ignore_index=True, )

Jumlah data duplikat: 5258


### **Step-3:** `Creating New Attributes`

Buat kolom bernama `FirstTransaction` dan isi dengan bulan pertama dari transaksi tiap customer.

In [79]:
df_merge['FirstTransaction'] = df_merge.groupby('Customer ID')['InvoiceDate'].transform('min')
df_merge['FirstTransactionMonth'] = df_merge['FirstTransaction'].dt.to_period('M')
df_merge.sample()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,FirstTransaction,FirstTransactionMonth
6671,538071,21888,BINGO SET,2,2010-12-09 14:09:00,7.62,21716,United Kingdom,2010-12-09 14:09:00,2010-12


Hitung total selisih bulan di antara `FirstTransaction` dengan `InvoiceDate`

In [80]:
df_merge['PeriodDistance'] = (df_merge['InvoiceDate'] - df_merge['FirstTransaction']).astype('timedelta64[M]').astype(int)
df_merge.sample()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,FirstTransaction,FirstTransactionMonth,PeriodDistance
461392,574684,21094,SET/6 RED SPOTTY PAPER PLATES,12,2011-11-06 12:56:00,0.85,17581,United Kingdom,2010-12-02 14:47:00,2010-12,11


Hitung revenue untuk setiap invoice dengan mengalikan `Quantity` dengan `Price`

In [81]:
df_merge['Revenue'] = df_merge['Price'] * df_merge['Quantity']
df_merge.sample()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,FirstTransaction,FirstTransactionMonth,PeriodDistance,Revenue
463572,574826,23308,SET OF 60 VINTAGE LEAF CAKE CASES,1,2011-11-07 10:58:00,0.55,15870,United Kingdom,2011-11-07 10:58:00,2011-11,0,0.55


### **Q1:** `Retention Rate`

Buat pivot table dengan menggunakan kolom `FirstTransactionMonth` dan `PeriodDistance`, kemudian hitung jumlah (distinct) dari `Customer ID`.

In [82]:
# Pivot table antara transaksi awal vs jarak transaksi
cohorts = df_merge.pivot_table(
    index='FirstTransactionMonth',
    columns='PeriodDistance',
    values='Customer ID',
    aggfunc='nunique',
)

# Ubah menjadi bentuk long format
cohorts = cohorts.stack().to_frame('Customer ID').reset_index()

# Tambahkan kolom persentase
max_value = cohorts.groupby('FirstTransactionMonth')['Customer ID'].transform('max')
cohorts['Pct Customer'] = cohorts['Customer ID'] / max_value

Visualisasikan `cohort retention` menggunakan HeatMap.

In [83]:
cohorts['FirstTransactionMonth'] = cohorts['FirstTransactionMonth'].astype(str)

# Plot dengan heat map
heatmap = alt.Chart(cohorts).mark_rect().encode(
    x=alt.X('PeriodDistance:O', title='Distance (month)').axis(labelAngle=0),
    y=alt.Y('FirstTransactionMonth:O', title='Cohort'),
    color=alt.Color('Pct Customer').legend(None),
).properties(
    width=800,
    height=300,
    title='Cohort Analysis - Retention'
)

# Menambahkan text annotations
text = heatmap.mark_text(baseline='middle').encode(
    text=alt.Text('Pct Customer', format='.2%'),
    color=alt.condition(
        alt.datum['Pct Customer'] == 1.0,
        alt.value('white'),
        alt.value('black')
    )
)

# Menambahkan judul
(heatmap + text).properties(
    title=alt.Title(
        "Customer Loyalty Currently Peaks at 42% and Requires Enhancement",
        subtitle='Monthly Cohorts by Retention Rate (%)',
        anchor='start',
        font='Calibri',
        fontSize=18,
        offset=20,
    ),
    width=700, height=250
)

alt.LayerChart(...)

___
### **Q2:** `Total Revenue Generated`

Buat pivot table dengan menggunakan kolom `FirstTransactionMonth` dan `PeriodDistance`, kemudian hitung jumlah dari `Revenue`.

In [84]:
# Pivot table antara transaksi awal vs jarak transaksi
cohorts = df_merge.pivot_table(
    index='FirstTransactionMonth',
    columns='PeriodDistance',
    values='Revenue',
    aggfunc='sum',
)

# Ubah menjadi bentuk long format
cohorts = cohorts.stack().to_frame('Revenue').reset_index()

Visualisasikan `cohort revenue` menggunakan HeatMap.

In [85]:
cohorts['FirstTransactionMonth'] = cohorts['FirstTransactionMonth'].astype(str)

# Plot dengan heat map
heatmap = alt.Chart(cohorts).mark_rect().encode(
    x=alt.X('PeriodDistance:O', title='Distance (month)').axis(labelAngle=0),
    y=alt.Y('FirstTransactionMonth:O', title='Cohort'),
    color=alt.Color('Revenue').legend(None),
).properties(
    width=800,
    height=300,
    title='Cohort Analysis - Revenue'
)

# Menambahkan text annotations
text = heatmap.mark_text(baseline='middle').encode(
    text=alt.Text('Revenue', format='$.2s'),
    color=alt.condition(
        alt.datum['Revenue'] > 800000,
        alt.value('white'),
        alt.value('black')
    )
)

# Menambahkan judul
(heatmap + text).properties(
    title=alt.Title(
        "In December 2011, Revenue Hit its Lowest Point",
        subtitle='Monthly Cohorts by Revenue (in dollars)',
        anchor='start',
        font='Calibri',
        fontSize=18,
        offset=20,
    ),
    width=700, height=250
)

alt.LayerChart(...)

___
### **Q3:** `Total Quantity Sold`

Buat pivot table dengan menggunakan kolom `FirstTransactionMonth` dan `PeriodDistance`, kemudian hitung jumlah dari `Quantity`.

In [86]:
# Pivot table antara transaksi awal vs jarak transaksi
cohorts = df_merge.pivot_table(
    index='FirstTransactionMonth',
    columns='PeriodDistance',
    values='Quantity',
    aggfunc='sum',
)

# Ubah menjadi bentuk long format
cohorts = cohorts.stack().to_frame('Quantity').reset_index()

Visualisasikan `cohort total quantity sold` menggunakan HeatMap.

In [87]:
cohorts['FirstTransactionMonth'] = cohorts['FirstTransactionMonth'].astype(str)

# Plot dengan heat map
heatmap = alt.Chart(cohorts).mark_rect().encode(
    x=alt.X('PeriodDistance:O', title='Distance (month)').axis(labelAngle=0),
    y=alt.Y('FirstTransactionMonth:O', title='Cohort'),
    color=alt.Color('Quantity').legend(None),
).properties(
    width=800,
    height=300,
    title='Cohort Analysis - Total Sold Quantity'
)

# Menambahkan text annotations
text = heatmap.mark_text(baseline='middle').encode(
    text=alt.Text('Quantity', format='.2s'),
    color=alt.condition(
        alt.datum['Quantity'] > 200000,
        alt.value('white'),
        alt.value('black')
    )
)

# Menambahkan judul
(heatmap + text).properties(
    title=alt.Title(
        "Customer Reluctance Towards Purchasing New Products is Increasing",
        subtitle='Monthly Cohorts by Total Sold Quantity (sales)',
        anchor='start',
        font='Calibri',
        fontSize=18,
        offset=20,
    ),
    width=700, height=250
)

alt.LayerChart(...)